In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = "E:\\MIMIC_III\\mimic-iii-clinical-database-1.4\\"

In [3]:
# load Patients and Diagnoses_icd
patients = pd.read_csv(path + '\\patients.csv')
diagnoses = pd.read_csv(path + '\\diagnoses_icd.csv')

In [4]:
print(f"Patients shape is {patients.shape}")
print(f"Diagnoses shape is {diagnoses.shape}")
print()

# Check if ICD codes are numeric or not
icd_column_dtype = diagnoses['ICD9_CODE'].dtype
print("Checking ICD codes:")
print(f"\tICD codes column dtype is {icd_column_dtype}")

# Check if all ICD codes are numeric
if pd.api.types.is_numeric_dtype(diagnoses['ICD9_CODE']):
    print("\tAll ICD codes are numeric.")
else:
    print("\tICD codes are not entirely numeric. Attempting to clean and convert...")

    # Attempt to clean and convert to numeric, ignoring errors
    diagnoses['ICD9_CODE'] = pd.to_numeric(diagnoses['ICD9_CODE'], errors='coerce')

    # Report how many could not be converted
    null_count = diagnoses['ICD9_CODE'].isnull().sum()
    print(f"\tConverted ICD codes to numeric. {null_count} rows could not be converted and contain NaN.")


Patients shape is (46520, 8)
Diagnoses shape is (651047, 5)

Checking ICD codes:
	ICD codes column dtype is object
	ICD codes are not entirely numeric. Attempting to clean and convert...
	Converted ICD codes to numeric. 97296 rows could not be converted and contain NaN.


In [5]:
# checking why ICD9_CODE columns is Object
diagnoses['ICD9_CODE'].isna().sum()

97296

In [6]:
diagnoses_cleaned = diagnoses.dropna(subset=['ICD9_CODE'])
# Verify the result
print(f"Diagnoses shape after dropping missing ICD9_CODE: {diagnoses_cleaned.shape}")

Diagnoses shape after dropping missing ICD9_CODE: (553751, 5)


In [7]:
# Check if ICD codes are numeric or not
icd_column_dtype = diagnoses_cleaned['ICD9_CODE'].dtype
print("Checking ICD codes:")
print(f"\tICD codes column dtype is {icd_column_dtype}")

# Check if all ICD codes are numeric
if pd.api.types.is_numeric_dtype(diagnoses['ICD9_CODE']):
    print("\tAll ICD codes are numeric.")
else:
    print("\tICD codes are not entirely numeric. Attempting to clean and convert...")

    # Attempt to clean and convert to numeric, ignoring errors
    diagnoses['ICD9_CODE'] = pd.to_numeric(diagnoses['ICD9_CODE'], errors='coerce')

    # Report how many could not be converted
    null_count = diagnoses['ICD9_CODE'].isnull().sum()
    print(f"\tConverted ICD codes to numeric. {null_count} rows could not be converted and contain NaN.")


Checking ICD codes:
	ICD codes column dtype is float64
	All ICD codes are numeric.


In [9]:
# now lets extract sepsis patients ( sepsis , severe sepsis , septic shock)
sepsis_codes = {
    99591.0: 'Sepsis',
    99592.0: 'Severe Sepsis',
    78552.0: 'Septic Shock'
}

In [13]:
# Filter for sepsis-related diagnoses
sepsis_diagnoses = diagnoses_cleaned[diagnoses_cleaned['ICD9_CODE'].isin(sepsis_codes.keys())]


In [14]:
sepsis_diagnoses.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object')

In [15]:
sepsis_diagnoses.shape

(7770, 5)

In [29]:
# Merge with the patients table to get demographic details
sepsis_patients = sepsis_diagnoses.merge(patients, on='SUBJECT_ID', how='inner')

In [30]:
print(sepsis_patients.head())
sepsis_patients.shape

   ROW_ID_x  SUBJECT_ID  HADM_ID  SEQ_NUM  ICD9_CODE  ROW_ID_y GENDER  \
0      1547         117   164853     16.0    99592.0       108      F   
1      1604         124   138376      6.0    99592.0       114      M   
2       505          64   172056      3.0    99591.0        57      F   
3       679          85   112077     18.0    99591.0        77      M   
4       131          21   111970      2.0    78552.0        18      M   

                   DOB                  DOD             DOD_HOSP  \
0  2083-12-28 00:00:00  2133-12-01 00:00:00  2133-12-01 00:00:00   
1  2090-11-19 00:00:00  2166-02-01 00:00:00  2166-02-01 00:00:00   
2  2116-06-27 00:00:00                  NaN                  NaN   
3  2090-09-18 00:00:00  2167-09-12 00:00:00  2167-09-12 00:00:00   
4  2047-04-04 00:00:00  2135-02-08 00:00:00  2135-02-08 00:00:00   

               DOD_SSN  EXPIRE_FLAG  
0  2133-12-01 00:00:00            1  
1  2166-02-01 00:00:00            1  
2                  NaN            0  


(7770, 12)